[View in Colaboratory](https://colab.research.google.com/github/novdov/study/blob/master/Digit.ipynb)

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [0]:
train = pd.read_csv("drive/Colab Notebooks/train_digit.csv")
test = pd.read_csv("drive/Colab Notebooks/test_digit.csv")
train2 = train.copy()

In [14]:
y_train = train["label"]
train = train.drop("label", axis=1)
X_train = train

X_train = X_train.values.reshape(-1, 28, 28, 1)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

y_train = pd.get_dummies(y_train)

y_train = y_train.values

yy = train2["label"]

yy_dummies = pd.get_dummies(yy)

label = yy_dummies.columns

X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # 32개의 커널을 가진 컨볼루션 계층을 만듦
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME') # 커널 슬라이딩 시 이미지의 가장 외곽에서 한 칸 밖으로 움직임
L1 = tf.nn.relu(L1)
# 풀링 계층
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], 
                    padding='SAME')

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], 
                    padding='SAME')

W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# epoch = 50
batch_size = 100

train_size = X_train.shape[0]
total_batch = int(train_size / batch_size)

# 시간이 오래 걸려 colab에서 실행
for epoch in range(100):
    total_cost = 0
    
    idx = np.random.permutation(train_size)
    X_random = X_train[idx]
    y_random = y_train[idx]
    
    for i in range(total_batch):
        batch_xs = X_random[i * total_batch:(i+1) * total_batch]
        batch_ys = y_random[i * total_batch:(i+1) * total_batch]
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, 
                                                             Y: batch_ys, 
                                                             keep_prob: 0.7})
        total_cost += cost_val        

    print("Epoch: {:04d}".format(epoch + 1), 
          "Avg. cost: {:.3f}".format(total_cost / total_batch))
    
print()
print("Optimization completed!")

X_test = test.values.reshape(-1, 28, 28, 1)

pred = tf.argmax(model, 1)

prediction = sess.run(pred, feed_dict={X: X_test, keep_prob: 1})

prediction = pd.Series(prediction, name="Label")

submission = pd.concat([pd.Series(range(1,28001), name = "ImageId"), prediction],axis = 1)

Epoch: 0001 Avg. cost: nan
Epoch: 0002 Avg. cost: nan
Epoch: 0003 Avg. cost: nan
Epoch: 0004 Avg. cost: nan
Epoch: 0005 Avg. cost: nan
Epoch: 0006 Avg. cost: nan
Epoch: 0007 Avg. cost: nan
Epoch: 0008 Avg. cost: nan
Epoch: 0009 Avg. cost: nan
Epoch: 0010 Avg. cost: nan
Epoch: 0011 Avg. cost: nan
Epoch: 0012 Avg. cost: nan
Epoch: 0013 Avg. cost: nan
Epoch: 0014 Avg. cost: nan
Epoch: 0015 Avg. cost: nan
Epoch: 0016 Avg. cost: nan
Epoch: 0017 Avg. cost: nan
Epoch: 0018 Avg. cost: nan
Epoch: 0019 Avg. cost: nan
Epoch: 0020 Avg. cost: nan
Epoch: 0021 Avg. cost: nan
Epoch: 0022 Avg. cost: nan
Epoch: 0023 Avg. cost: nan
Epoch: 0024 Avg. cost: nan
Epoch: 0025 Avg. cost: nan
Epoch: 0026 Avg. cost: nan
Epoch: 0027 Avg. cost: nan
Epoch: 0028 Avg. cost: nan
Epoch: 0029 Avg. cost: nan
Epoch: 0030 Avg. cost: nan
Epoch: 0031 Avg. cost: nan
Epoch: 0032 Avg. cost: nan
Epoch: 0033 Avg. cost: nan
Epoch: 0034 Avg. cost: nan
Epoch: 0035 Avg. cost: nan
Epoch: 0036 Avg. cost: nan
Epoch: 0037 Avg. cost: nan
E

Epoch: 0086 Avg. cost: nan
Epoch: 0087 Avg. cost: nan
Epoch: 0088 Avg. cost: nan
Epoch: 0089 Avg. cost: nan
Epoch: 0090 Avg. cost: nan
Epoch: 0091 Avg. cost: nan
Epoch: 0092 Avg. cost: nan
Epoch: 0093 Avg. cost: nan
Epoch: 0094 Avg. cost: nan
Epoch: 0095 Avg. cost: nan
Epoch: 0096 Avg. cost: nan
Epoch: 0097 Avg. cost: nan
Epoch: 0098 Avg. cost: nan
Epoch: 0099 Avg. cost: nan
Epoch: 0100 Avg. cost: nan

Optimization completed!


In [0]:
submission.to_csv("tensorflow_cnn.csv", index=False)

In [0]:
from google.colab import files

with open('tensorflow_cnn.csv', 'w') as f:
    f.write(submission)

files.download('example.txt')

In [0]:
from google.colab import files
files.download('tensorflow_cnn.csv')